In [1]:
import numpy as np
import sympy as sp
from numpy import sin, cos, pi

from scipy.spatial.transform import Rotation as R

In [2]:
def DH_trans(DH, joint_val):

    d, theta, a, alpha = (0,0,0,0)

    if (DH[0] == 'r'):

        d, theta, a, alpha = (DH[1], joint_val, DH[2], DH[3])

    elif (DH[0] == 'p'):

        d, theta, a, alpha = (joint_val, DH[1], DH[2], DH[3])

    elif (DH[0] == 'f'):

        d, theta, a, alpha = (DH[1], DH[2], DH[3], DH[4])

    trans_mat = np.array([[cos(theta), -sin(theta)*cos(alpha), sin(theta)*sin(alpha),    a*cos(theta)],
                          [sin(theta), cos(theta)*cos(alpha),    -cos(theta)*sin(alpha), a*sin(theta)],
                          [0,          sin(alpha),               cos(alpha),               d           ],
                          [0,          0,                        0,                        1           ]])

    return trans_mat


def joint_transforms(DH_Params):

    transforms = []

    current_DOF = 0

    transforms.append(np.eye(4))

    for DH in DH_Params:

        if (DH[0] == 'r'):
            transforms.append(DH_trans(DH, 0.0))
            current_DOF = current_DOF + 1
        elif (DH[0] == 'p'):
            transforms.append(DH_trans(DH, 0))
            current_DOF = current_DOF + 1
        else:
            transforms.append(DH_trans(DH, 0.0))

    return transforms

def joint_frames(transforms):
         
    joint_frames = [transforms[0]]
    for trans in transforms[1:]:
        joint_frames.append(joint_frames[-1].dot(trans))

    return joint_frames
def euler_angles(pose):
    if pose[0, 2] == 1:
        theta_1, theta_2, theta_3 = np.arctan2(pose[1, 0], pose[1, 1]), pi/2, 0
    elif pose[0, 2] == -1:
        theta_1, theta_2, theta_3 = -np.arctan2(pose[1, 0], pose[1, 1]), pi/2, 0
    else:
        theta_1 = np.arctan2(-pose[1, 2], pose[2, 2])
        theta_2 = np.arcsin(pose[0, 2])
        theta_3 = np.arctan2(-pose[0, 1], pose[0, 0])
    return (theta_1, theta_2, theta_3)
       
def rotation_matrix_from_euler_angles(euler_angles, sequence):
    """
    Return a transformation matrix for the given euler angles.
    
    :param euler_angles: are the euler angles.
    :type euler_angles: Matrix
    ...
    :param sequence: is the euler angles configuration.
    :type sequence: string 
    ...
    raise ValueError: if an invalid or unsupported euler angle sequence is received.
    ...
    :return: a transformation matrix for the given euler angles.
    """
    sequence = sequence.lower()
    if not all([char in 'xyz' for char in sequence]) or len(sequence) != 3:
        raise ValueError(f'Invalid or unsupported euler angle sequence {sequence}.')
    T1 = hrotation3(sequence[0], euler_angles[0])
    T2 = hrotation3(sequence[1], euler_angles[1])
    T3 = hrotation3(sequence[2], euler_angles[2])
    return T1 * T2 * T3
def hrotation3(axis, angle):
    """
    Return a transformation matrix in 3D homogeneous coordinates that represents the rotation by `angle` around `axis`.
    
    :param axis: is the axis of rotation. It should be one of ['x', 'y', 'z', 'n', 'o', 'a'].
    :type axis: string
    ...
    :param angle: is the angle of rotation.
    :type angle: sympy expression, sympy symbol or a number
    ...
    raise ValueError: if an invalid axis value is received.
    ...
    :return: the rotation matrix
    """
    if axis in ['x', 'n']:
        return sp.Matrix([
            [1, 0, 0, 0],
            [0, sp.cos(angle), -sp.sin(angle), 0], 
            [0, sp.sin(angle), sp.cos(angle), 0],
            [0, 0, 0, 1]
        ])
    elif axis in ['y', 'o']:
        return sp.Matrix([
            [sp.cos(angle), 0, sp.sin(angle), 0],
            [0, 1, 0, 0],
            [-sp.sin(angle), 0, sp.cos(angle), 0],
            [0, 0, 0, 1]
        ])
    elif axis in ['z', 'a']:
        return sp.Matrix([
            [sp.cos(angle), -sp.sin(angle), 0, 0], 
            [sp.sin(angle), sp.cos(angle), 0, 0], 
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
    else:
        raise ValueError(f'Expected one of [x, y, z, n, o, a] but received {axis}.') 

# DH Parameter Layout:
# ['r', d, a, alpha] for revolute joints
# ['p', theta, a, alpha] for prismatic joints
# ['f', d, theta, a, alpha] for fixed joints
def htranslation3(x=0, y=0, z=0):
    """Return transformation matrix in 3D homogeneous coordinates with embedded translation."""
    return sp.Matrix([[1, 0, 0, x],
                      [0, 1, 0, y],
                      [0, 0, 1, z],
                      [0, 0, 0, 1]])

In [3]:
def xml_string(DH_Params, scale=1):

    outstring = "<!-- DH Parameters \n"
    for DH_param in DH_Params:
        outstring = outstring + "\t{}, {}\n".format(DH_param[0],str(DH_param[1:])[1:-1])
    outstring = outstring + "-->\n"

    transforms = joint_transforms(DH_Params)
    frames = joint_frames(transforms)
    outstring = outstring + "<robot name='robot'>\n"
    outstring = outstring + "\t<material name='blue'>\n\t\t<color rgba='0.6 0.6 0.6 1'/>\n\t</material>\n"
    outstring = outstring + "\t<material name='red'>\n\t\t<color rgba='1 1 1 1'/>\n\t</material>\n"

    for i in range(len(transforms) - 1):

        el = transforms[i]
        fr = frames[i]

        # We need to create a cylinder to represent the joint
        # If the index is not zero, connect it to the previous link
        # And a joint to connect it to the link
        # And a box to connect the joints

        rpy = euler_angles(el)
        outstring = outstring + "\t<link name='a{}'>\n".format(i)
        outstring = outstring + "\t\t<visual>\n"
        outstring = outstring + "\t\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(rpy[0], rpy[1], rpy[2], el[0,3], el[1,3], el[2,3])
        outstring = outstring + "\t\t\t<geometry>\n"
        outstring = outstring + "\t\t\t\t<cylinder length='0.7' radius='0.43'/>\n"
        outstring = outstring + "\t\t\t</geometry>\n"
        outstring = outstring + "\t\t\t<material name='blue'/>\n"
        outstring = outstring + "\t\t</visual>\n"
        outstring = outstring + "\t</link>\n"

        # If not on the first transformation, fix the cylinder to the previous link
        if(i != 0):
            outstring = outstring + "\t<joint name='fix_a{}_to_l{}' type='fixed'>\n".format(i, i-1)
            outstring = outstring + "\t\t<parent link='l{}'/>\n".format(i-1)
            outstring = outstring + "\t\t<child link='a{}'/>\n".format(i)
            outstring = outstring + "\t\t<origin rpy='0 0 0' xyz='0 0 0'/>\n"
            outstring = outstring + "\t</joint>\n"

        # Add a cylinder that goes from the current origin to the next one
        origins_vector = transforms[i + 1][0:3,3]
        print(origins_vector)
        origins_vector_norm = np.linalg.norm(origins_vector)

        cylinder_origin = origins_vector/2

        rpy = [0, 0, 0]

        if (origins_vector_norm != 0.0):

            origins_vector_unit = origins_vector/origins_vector_norm

            axis = np.cross(origins_vector, np.array([0, 0, -1]))

            axis_norm = np.linalg.norm(axis)
            if (axis_norm != 0.0):
                axis = axis/np.linalg.norm(axis)

            angle = np.arccos(origins_vector_unit @ np.array([0, 0, 1]))

            rpy = R.from_rotvec(angle * axis).as_euler('XYZ')
            rpy = angle*axis

        outstring = outstring + "\t<link name='l{}'>\n".format(i)
        outstring = outstring + "\t\t<visual>\n"
        outstring = outstring + "\t\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(rpy[0], rpy[1], rpy[2], cylinder_origin[0], cylinder_origin[1], cylinder_origin[2])
        outstring = outstring + "\t\t\t<geometry>\n"
        outstring = outstring + "\t\t\t\t<cylinder length='{}' radius='0.3'/>\n".format(origins_vector_norm) 
        outstring = outstring + "\t\t\t</geometry>\n"
        outstring = outstring + "\t\t\t<material name='red'/>\n"
        outstring = outstring + "\t\t</visual>\n"
        if (DH_Params[i][0] == 'p'): 
            #rpy = euler_angles(transforms[i-1]) 
            rpy = [0, 0, 0]
            a = np.concatenate((fr[0:3,0:3],np.zeros(shape = (1,3), dtype = 'int')),axis=0)
            rotm = np.concatenate((a, np.array([[0],[0],[0],[1]]) ),axis=1)
            rotms = rotation_matrix_from_euler_angles(sp.Matrix([[rpy[0]],[rpy[1]],[rpy[2]]]),'XYZ')
            posm = htranslation3( fr[0,3], fr[1,3], fr[2,3])
            ofsm = rotms * htranslation3(0,0,1.725)
            trs= ofsm*rotms
            
            outstring = outstring + "\t\t<visual>\n"
            outstring = outstring + "\t\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(rpy[0], rpy[1], rpy[2],0,0,-1.725)
            outstring = outstring + "\t\t\t<geometry>\n"
            outstring = outstring + "\t\t\t\t<cylinder length='4' radius='0.3'/>\n"
            outstring = outstring + "\t\t\t</geometry>\n"
            outstring = outstring + "\t\t</visual>\n"
        outstring = outstring + "\t</link>\n"

        # Add the actual joint between the cylinder and link

        jointType = ""

        if (DH_Params[i][0] == 'r'):
            jointType = "revolute"
        elif (DH_Params[i][0] == 'p'):
            jointType = "prismatic"
        else:
            jointType = "fixed"

        outstring += "\t<joint name='move_l{}_from_a{}' type='{}'>\n".format(i, i, jointType)
        outstring += "\t\t<parent link='a{}'/>\n".format(i)
        outstring += "\t\t<child link='l{}'/>\n".format(i)
        outstring += "\t\t<axis xyz='0 0 1'/>\n"
        if (DH_Params[i-1][0] == 'r'):
            outstring = outstring + "\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(0 if i == 0 else DH_Params[i-1][3],0 , 0,el[0,3], el[1,3], el[2,3])  
        elif (DH_Params[i-1][0] == 'p'):
            outstring = outstring + "\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(0 if i == 0 else DH_Params[i-1][3],0 , 0 if i == 0 else DH_Params[i-1][1],el[0,3], el[1,3], el[2,3])  
        else:
            outstring = outstring + "\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(0 if i == 0 else DH_Params[i-1][4],0 , 0 if i == 0 else DH_Params[i-1][2],el[0,3], el[1,3], el[2,3])  
       
        if (DH_Params[i][0] == 'p'):
            outstring = outstring + "\t\t<limit effort='12' lower='1' upper='4' velocity='2.6100'/>\n" 
        else:  
            outstring = outstring + "\t\t<limit effort='12' lower='-2.9671' upper='2.9671' velocity='2.6100'/>\n" 
        
        outstring = outstring + "\t</joint>\n" 

    outstring = outstring + "</robot>\n"
    return outstring

In [4]:
def xml_string(DH_Params, scale=1):

    outstring = "<!-- DH Parameters \n"
    for DH_param in DH_Params:
        outstring = outstring + "\t{}, {}\n".format(DH_param[0],str(DH_param[1:])[1:-1])
    outstring = outstring + "-->\n"

    transforms = joint_transforms(DH_Params)
    frames = joint_frames(transforms)
    outstring = outstring + "<robot name='robot'>\n"
    outstring = outstring + "\t<material name='blue'>\n\t\t<color rgba='0.6 0.6 0.6 1'/>\n\t</material>\n"
    outstring = outstring + "\t<material name='red'>\n\t\t<color rgba='1 1 1 1'/>\n\t</material>\n"

    for i in range(len(transforms) - 1):

        el = transforms[i]
        fr = frames[i]

        # We need to create a cylinder to represent the joint
        # If the index is not zero, connect it to the previous link
        # And a joint to connect it to the link
        # And a box to connect the joints

        rpy = euler_angles(el)
        outstring = outstring + "\t<link name='a{}'>\n".format(i)
        outstring = outstring + "\t\t<visual>\n"
        outstring = outstring + "\t\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(rpy[0], rpy[1], rpy[2], el[0,3], el[1,3], el[2,3])
        outstring = outstring + "\t\t\t<geometry>\n"
        outstring = outstring + "\t\t\t\t<cylinder length='0.7' radius='0.43'/>\n"
        outstring = outstring + "\t\t\t</geometry>\n"
        outstring = outstring + "\t\t\t<material name='blue'/>\n"
        outstring = outstring + "\t\t</visual>\n"
        outstring = outstring + "\t</link>\n"

        # If not on the first transformation, fix the cylinder to the previous link
        if(i != 0):
            outstring = outstring + "\t<joint name='fix_a{}_to_l{}' type='fixed'>\n".format(i, i-1)
            outstring = outstring + "\t\t<parent link='l{}'/>\n".format(i-1)
            outstring = outstring + "\t\t<child link='a{}'/>\n".format(i)
            outstring = outstring + "\t\t<origin rpy='0 0 0' xyz='0 0 0'/>\n"
            outstring = outstring + "\t</joint>\n"

        # Add a cylinder that goes from the current origin to the next one
        origins_vector = transforms[i + 1][0:3,3]
        print(origins_vector)
        origins_vector_norm = np.linalg.norm(origins_vector)

        cylinder_origin = origins_vector/2

        rpy = [0, 0, 0]

        if (origins_vector_norm != 0.0):

            origins_vector_unit = origins_vector/origins_vector_norm

            axis = np.cross(origins_vector, np.array([0, 0, -1]))

            axis_norm = np.linalg.norm(axis)
            if (axis_norm != 0.0):
                axis = axis/np.linalg.norm(axis)

            angle = np.arccos(origins_vector_unit @ np.array([0, 0, 1]))

            rpy = R.from_rotvec(angle * axis).as_euler('XYZ')
            rpy = angle*axis

        outstring = outstring + "\t<link name='l{}'>\n".format(i)
        outstring = outstring + "\t\t<visual>\n"
        outstring = outstring + "\t\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(rpy[0], rpy[1], rpy[2], cylinder_origin[0], cylinder_origin[1], cylinder_origin[2])
        outstring = outstring + "\t\t\t<geometry>\n"
        outstring = outstring + "\t\t\t\t<cylinder length='{}' radius='0.3'/>\n".format(origins_vector_norm) 
        outstring = outstring + "\t\t\t</geometry>\n"
        outstring = outstring + "\t\t\t<material name='red'/>\n"
        outstring = outstring + "\t\t</visual>\n"
        if (DH_Params[i][0] == 'p'): 
            #rpy = euler_angles(transforms[i-1]) 
            rpy = [0, 0, 0]
            a = np.concatenate((fr[0:3,0:3],np.zeros(shape = (1,3), dtype = 'int')),axis=0)
            rotm = np.concatenate((a, np.array([[0],[0],[0],[1]]) ),axis=1)
            rotms = rotation_matrix_from_euler_angles(sp.Matrix([[rpy[0]],[rpy[1]],[rpy[2]]]),'XYZ')
            posm = htranslation3( fr[0,3], fr[1,3], fr[2,3])
            ofsm = rotms * htranslation3(0,0,1.725)
            trs= ofsm*rotms
            
            outstring = outstring + "\t\t<visual>\n"
            outstring = outstring + "\t\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(rpy[0], rpy[1], rpy[2],0,0,-1.725)
            outstring = outstring + "\t\t\t<geometry>\n"
            outstring = outstring + "\t\t\t\t<cylinder length='4' radius='0.3'/>\n"
            outstring = outstring + "\t\t\t</geometry>\n"
            outstring = outstring + "\t\t</visual>\n"
        outstring = outstring + "\t</link>\n"

        # Add the actual joint between the cylinder and link

        jointType = ""

        if (DH_Params[i][0] == 'r'):
            jointType = "revolute"
        elif (DH_Params[i][0] == 'p'):
            jointType = "prismatic"
        else:
            jointType = "fixed"

        outstring += "\t<joint name='move_l{}_from_a{}' type='{}'>\n".format(i, i, jointType)
        outstring += "\t\t<parent link='a{}'/>\n".format(i)
        outstring += "\t\t<child link='l{}'/>\n".format(i)
        outstring += "\t\t<axis xyz='0 0 1'/>\n"
        if (DH_Params[i-1][0] == 'r'):
            outstring = outstring + "\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(0 if i == 0 else DH_Params[i-1][3],0 , 0,el[0,3], el[1,3], el[2,3])  
        elif (DH_Params[i-1][0] == 'p'):
            outstring = outstring + "\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(0 if i == 0 else DH_Params[i-1][3],0 , 0 if i == 0 else DH_Params[i-1][1],el[0,3], el[1,3], el[2,3])  
        else:
            outstring = outstring + "\t\t<origin rpy='{} {} {}' xyz='{} {} {}'/>\n".format(0 if i == 0 else DH_Params[i-1][4],0 , 0 if i == 0 else DH_Params[i-1][2],el[0,3], el[1,3], el[2,3])  
       
        if (DH_Params[i][0] == 'p'):
            outstring = outstring + "\t\t<limit effort='12' lower='1' upper='4' velocity='2.6100'/>\n" 
        else:  
            outstring = outstring + "\t\t<limit effort='12' lower='-2.9671' upper='2.9671' velocity='2.6100'/>\n" 
        
        outstring = outstring + "\t</joint>\n" 

    outstring = outstring + "</robot>\n"
    return outstring

In [6]:
from packages.serial_link_robot import *

In [7]:
theta1, theta2, D3, d1, k1, k2, k3, d3 = sp.symbols('theta1, theta2, D3, d1, k1, k2, k3, d3')
robot = SerialLinkRobot()
robot.add_revolute_joint(theta1, d1, 0, sp.pi/2)
robot.add_revolute_joint(theta2, 0, k1, sp.pi/2)
robot.add_prismatic_joint(0, D3, 0, 0)
robot.add_subs([(d1, 1), (k1, 1)])

In [8]:
robot.interact()

[('r', theta1, d1, 0, pi/2), ('r', theta2, 0, k1, pi/2), ('p', 0, D3, 0, 0)]


In [19]:
robot.get_dh_table()

Matrix([
[theta1, d1,  0, pi/2],
[theta2,  0, k1, pi/2],
[     0, D3,  0,    0]])

In [12]:
from packages.dh2urdf import *

In [13]:
# DH Parameter Layout:
# ['r', d, a, alpha] for revolute joints
# ['p', theta, a, alpha] for prismatic joints
# ['f', d, theta, a, alpha] for fixed joints
DH_Params = []
constraints=[]
DH_Params.append(['r', 2, 0, 3, 0])
constraints.append([12,-pi,pi,2.6])
DH_Params.append(['r', 0, 0, 2,pi/2])
constraints.append([12,-pi,pi,2.6])
DH_Params.append(['p', 0, 0, 2,0])
constraints.append([12,1,4,2.6])
DH_Params.append(['r', 0,0, 2, 0])
constraints.append([12,-pi,pi,2.6])
dh2urdf = DH2Urdf(DH_Params,constraints)

In [14]:
dh2urdf.save_urdf('outfile')

[3 0 2] 0
[2 0 0] 1
[2 0 0] 2
[2 0 0] 3


In [ ]:
import pybullet as p
import pybullet_data as pd
import math
import time
import numpy as np
import pybullet_robots.panda.panda_sim as panda_sim

p.connect(p.GUI)
p.configureDebugVisualizer(p.COV_ENABLE_Y_AXIS_UP,1)
p.setAdditionalSearchPath(pd.getDataPath())

timeStep=1./60.
p.setTimeStep(timeStep)
p.setGravity(0,-9.8,0)
 
panda = panda_sim.PandaSim(p,[0,0,0])
while (1):
	panda.step()
	p.stepSimulation()
	time.sleep(timeStep)
	


In [8]:
f = open("outfile.urdf", "w")

DH_Params = []
DH_Params.append(['r', 2, 3, 0])
DH_Params.append(['r', 0, 2,pi/2])
DH_Params.append(['p', 0, 2,0])
DH_Params.append(['r', 0, 2, 0])

f.write(xml_string(DH_Params))

f.close()

[3. 0. 2.]
[2. 0. 0.]
[2. 0. 0.]
[2. 0. 0.]


C:\Users\vase_\AppData\Local\Temp\ipykernel_2208\417944361.py:64: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  rpy = R.from_rotvec(angle * axis).as_euler('XYZ')


In [14]:
f = open("outfile.urdf", "w")
# DH Parameter Layout:
# ['r', d, a, alpha] for revolute joints
# ['p', theta, a, alpha] for prismatic joints
# ['f', d, theta, a, alpha] for fixed joints
DH_Params = []
DH_Params.append(['r', 2, 0, 0])
DH_Params.append(['p', 0, 0, -pi/2])
DH_Params.append(['p', 0, 0, 0])
DH_Params.append(['r', 0, 0, pi/2])
DH_Params.append(['r', 0, 0, -pi/2])
DH_Params.append(['r', 0, 2, 0])
#DH_Params.append(['r', 0, 2, 0])
#DH_Params.append(['p', 0, 0, 0])
#DH_Params.append(['f', 0, 0, 2, 0])
#DH_Params.append(['r', 0, 2, 0])

f.write(xml_string(DH_Params))

f.close()

axis is [-0.  0.  0.]
angle is 0.0
axis is [-0.  1.  0.]
angle is 1.5707963267948966


C:\Users\vase_\AppData\Local\Temp\ipykernel_9744\3612516681.py:67: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  rpy = R.from_rotvec(angle * axis).as_euler('XYZ')


In [26]:
f = open("outfile.urdf", "w")

DH_Params = []
DH_Params.append(['r', 1, 2, 0])
DH_Params.append(['r', 1, 3, 0])
DH_Params.append(['p', 0, 0, 0])
DH_Params.append(['f', 1, pi/2, 3, pi/2])
DH_Params.append(['r', 0, 2, 0])

f.write(xml_string(DH_Params))

f.close()


axis is [-0.  1.  0.]
angle is 1.1071487177940904
axis is [-0.  1.  0.]
angle is 1.2490457723982544
axis is [-1.000000e+00  6.123234e-17  0.000000e+00]
angle is 1.2490457723982544
axis is [-0.  1.  0.]
angle is 1.5707963267948966


C:\Users\vase_\AppData\Local\Temp\ipykernel_9744\1562938364.py:67: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  rpy = R.from_rotvec(angle * axis).as_euler('XYZ')


In [12]:
f = open("outfile.urdf", "w")

DH_Params = []
DH_Params.append(['r', 2.45, 1.50, -pi/2])
DH_Params.append(['r', 0, 3.00, 0])
DH_Params.append(['r', 0, 1.50, -pi/2])
DH_Params.append(['r', 1.38, 0, pi/2])
DH_Params.append(['r', 0, 0, pi/2])
DH_Params.append(['r', 2.00, 0, 0])

f.write(xml_string(DH_Params))

f.close()


axis is [-0.  1.  0.]
angle is 0.549374484771551
axis is [-0.  1.  0.]
angle is 1.5707963267948966
axis is [-0.  1.  0.]
angle is 1.5707963267948966
axis is [-0.  0.  0.]
angle is 0.0
axis is [-0.  0.  0.]
angle is 0.0


C:\Users\vase_\AppData\Local\Temp\ipykernel_9744\3612516681.py:67: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  rpy = R.from_rotvec(angle * axis).as_euler('XYZ')


In [ ]:
f = open("outfile.urdf", "w")
# DH Parameter Layout:
# ['r', d, a, alpha] for revolute joints
# ['p', theta, a, alpha] for prismatic joints
# ['f', d, theta, a, alpha] for fixed joints

DH_Params = []
DH_Params.append(['p', 0, 0, 0])
DH_Params.append(['r', 0, 2, 0])
DH_Params.append(['r', 0, 2, pi/2])
DH_Params.append(['p', 0, 0, 0])
#DH_Params.append(['r', 0, 0, 0])
f.write(xml_string(DH_Params))

f.close()